In [1]:
import dataset as ds
import utils
from torch.utils.data import DataLoader
import torch
import datetime
import transformer_timeseries as tst
import numpy as np

In [2]:
# * Configuration

## Hyperparams
test_size = 0.1
batch_size = 128
target_col_name = "tomorrow"
# timestamp_col = "timestamp"
# Only use data from this date and onwards
cutoff_date = datetime.datetime.now() - datetime.timedelta(days=2*365) # 2 years ago

related_ticker_names = [
    # "SPY", 
    # "QQQ", 
    # "ITOT", 
    # "VTI", 
    # "AMZN", 
    # "AAPL", 
    # "AMC", 
    # "NVDA", 
    # "MSFT", 
    # "ROKU", 
    # "GOOGL", 
    # "NFLX",
    # "DOGE-USD",
    # "BTC-USD",
    # "ETH-USD"
]
# related_ticker_predictor_fields = ["open", "high", "low", "close", "volume"]
related_ticker_predictor_fields = ["close", "volume"]
# predictors = ["close", "volume", "open", "high", "low", "edit_count", "sentiment", "neg_sentiment"]
predictors = ["close", "sentiment"]
for i in range(0,len(related_ticker_names)):
    for related_predictor in related_ticker_predictor_fields:
        predictors.append(f"{related_ticker_names[i]}_{related_predictor}")

In [3]:
## Params
dim_val = 512 # This can be any value divisible by n_heads. 512 is used in the original transformer paper.
n_heads = 8 # The number of attention heads (aka parallel attention layers). dim_val must be divisible by this number
n_decoder_layers = 4 # Number of times the decoder layer is stacked in the decoder
n_encoder_layers = 4 # Number of times the encoder layer is stacked in the encoder
dec_seq_len = 92 # length of input given to decoder
enc_seq_len = 153 # length of input given to encoder
output_sequence_length = 7 # target sequence length. If hourly data and length = 48, you predict 2 days ahead
window_size = enc_seq_len + output_sequence_length # used to slice data into sub-sequences
step_size = 1 # Step size, i.e. how many time steps does the moving window move at each step
in_features_encoder_linear_layer = 2048
in_features_decoder_linear_layer = 2048
max_seq_len = enc_seq_len
batch_first = True

In [4]:
# * Define input variables 
exogenous_vars = predictors # should contain strings. Each string must correspond to a column name
input_variables = [target_col_name] + exogenous_vars
# target_idx = -2 # ? not used? - index position of target in batched trg_y 

input_size = len(input_variables)

In [5]:
# * Read data
data = utils.read_data()

Reading file in data/tsla_historical_data.csv


In [6]:
# * Clean data
data = data.copy()[input_variables]

In [7]:
# * Remove test data from dataset
training_data = data[:-(round(len(data)*test_size))]

In [8]:
# * Make list of (start_idx, end_idx) pairs that are used to slice the time series 
## sequence into chunks. 
## Should be training data indices only
training_indices = utils.get_indices_entire_sequence(
    data=training_data, 
    window_size=window_size, 
    step_size=step_size)

In [9]:
# * Making instance of custom dataset class
training_data = ds.TransformerDataset(
    data=torch.tensor(training_data[input_variables].values).float(),
    indices=training_indices,
    enc_seq_len=enc_seq_len,
    dec_seq_len=dec_seq_len,
    target_seq_len=output_sequence_length
    )

From get_src_trg: data size = torch.Size([453, 3])


In [10]:
# * Making dataloader
training_data = DataLoader(training_data, batch_size)

i, batch = next(enumerate(training_data))

src, trg, trg_y = batch

In [11]:
# * Permute from shape [batch size, seq len, num features] to [seq len, batch size, num features]
if batch_first == False:

    shape_before = src.shape
    src = src.permute(1, 0, 2)
    print("src shape changed from {} to {}".format(shape_before, src.shape))

    shape_before = trg.shape
    trg = trg.permute(1, 0, 2)
    print("src shape changed from {} to {}".format(shape_before, src.shape))

In [12]:
print('input_size', input_size)
print('enc_seq_len', enc_seq_len)
print('batch_first', batch_first)

input_size 3
enc_seq_len 153
batch_first True


In [ ]:
# * Create Model
model = tst.TimeSeriesTransformer(
    input_size=input_size,
    dec_seq_len=enc_seq_len,
    batch_first=batch_first,
    num_predicted_features=1
    )

In [ ]:
# * Make src mask for decoder with size:
## [batch_size*n_heads, output_sequence_length, enc_seq_len]
src_mask = utils.generate_square_subsequent_mask(
    dim1=output_sequence_length,
    dim2=enc_seq_len
    )

In [ ]:
# * Make tgt mask for decoder with size:
## [batch_size*n_heads, output_sequence_length, output_sequence_length]
tgt_mask = utils.generate_square_subsequent_mask( 
    dim1=output_sequence_length,
    dim2=output_sequence_length
    )

In [ ]:
# * Generate output
output = model(
    src=src,
    tgt=trg,
    src_mask=src_mask,
    tgt_mask=tgt_mask
    )

In [ ]:
print(output)